# Big Data in Banking and Finance Exam 2025

## Claudine Linda Wa Nciko
## Student number: 169375





## QUESTION 1

## Introduction
In this notebook, we explore a dataset consisting of transaction and customer information in the banking and finance sector. Using PySpark, we perform several steps to gain insights into customer spending patterns. Specifically, we load two datasets transactions and customer details into PySpark DataFrames, convert appropriate columns to correct data types, and then join the datasets on the customer_id column. Afterward, we calculate the average transaction amount per customer and classify each customer as a low, moderate, or high spender based on their average spending. The final DataFrame containing the customer IDs, risk segments, average transaction amounts, and spend categories is saved in Parquet format for further analysis.

In [1]:
# Install PySpark
!pip install pyspark

# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, count, mean , desc, when , avg
from pyspark.sql.types import DateType, DoubleType
import requests
import os

## Spark Session Initialization
The first step in PySpark programming is to create a Spark session.  Our dispersed computations have a unified context thanks to this session.

In [2]:
# Initialize session
spark = SparkSession.builder.appName("Customer Transaction").getOrCreate()

## Load the datasets

In [3]:
# Load the transactions and customers Datasets
transactions_df = spark.read.csv("/content/transactions.csv", header=True, inferSchema=True)
customers_df = spark.read.csv("/content/customers.csv", header=True, inferSchema=True)

# Display a sample from transactions dataset
print("Transactions Schema:")
transactions_df.printSchema()
transactions_df.show(5)

# Display a sample from customers dataset
print("Customers Schema:")
customers_df.printSchema()
customers_df.show(5)


Transactions Schema:
root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- amount: double (nullable = true)
 |-- transaction_type: string (nullable = true)

+--------------+-----------+----------------+-------+----------------+
|transaction_id|customer_id|transaction_date| amount|transaction_type|
+--------------+-----------+----------------+-------+----------------+
|     TXN207175|   CUST0011|      2023-05-23| 785.84|          credit|
|     TXN733052|   CUST0003|      2023-07-14| 492.61|          credit|
|     TXN496922|   CUST0009|      2023-01-23|3651.36|           debit|
|     TXN705397|   CUST0003|      2023-10-10|1472.96|          credit|
|     TXN910620|   CUST0007|      2023-12-27| 357.08|           debit|
+--------------+-----------+----------------+-------+----------------+
only showing top 5 rows

Customers Schema:
root
 |-- customer_id: string (nullable = true)
 |-- account_open_date

## Exploratory Data Analysis (EDA)


Summary statistics for each dataset

In [4]:
# Summary statistics for transactions
print("\nSummary Statistics for transactions:")
transactions_df.describe().show()

# Summary statistics for customers
print("\nSummary Statistics for cutsomers:")
customers_df.describe().show()


Summary Statistics for transactions:
+-------+--------------+-----------+------------------+----------------+
|summary|transaction_id|customer_id|            amount|transaction_type|
+-------+--------------+-----------+------------------+----------------+
|  count|           200|        200|               200|             200|
|   mean|          NULL|       NULL|2392.4248999999995|            NULL|
| stddev|          NULL|       NULL|1417.5792633356295|            NULL|
|    min|     TXN113266|   CUST0001|             56.48|          credit|
|    max|     TXN994905|   CUST0020|           4980.65|           debit|
+-------+--------------+-----------+------------------+----------------+


Summary Statistics for cutsomers:
+-------+-----------+------------+
|summary|customer_id|risk_segment|
+-------+-----------+------------+
|  count|         20|          20|
|   mean|       NULL|        NULL|
| stddev|       NULL|        NULL|
|    min|   CUST0001|        high|
|    max|   CUST0020|   

The summary statistics show there are 200 transactions with an average amount of 2392.42, ranging from 56.48 to 4980.65.
The customer dataset contains 20 unique customers, each assigned a risk segment (low, medium, or high).

Covert data types
Even though the data is already inferred correctly, we include this block for clarity the data types.

In [5]:
# Convert 'transaction_date' to DateType and 'amount' to DoubleType explicitly in transactions_df
transactions_df = transactions_df.withColumn("transaction_date", col("transaction_date").cast(DateType())) \
                                 .withColumn("amount", col("amount").cast(DoubleType()))

# Convert 'account_open_date' to DateType in customers_df
customers_df = customers_df.withColumn("account_open_date", col("account_open_date").cast(DateType()))

# Verify the conversion
transactions_df.printSchema()
customers_df.printSchema()


root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- amount: double (nullable = true)
 |-- transaction_type: string (nullable = true)

root
 |-- customer_id: string (nullable = true)
 |-- account_open_date: date (nullable = true)
 |-- risk_segment: string (nullable = true)



##  Join the Datasets

Perform an inner join on the customer_id column to combine transaction details with customer information.

In [6]:
# Join Datasets on customer_id in customer transaction (custrans)
custrans = transactions_df.join(customers_df, on="customer_id", how="inner")

# Display the joined DataFrame to verify the join
custrans.show(10)


+-----------+--------------+----------------+-------+----------------+-----------------+------------+
|customer_id|transaction_id|transaction_date| amount|transaction_type|account_open_date|risk_segment|
+-----------+--------------+----------------+-------+----------------+-----------------+------------+
|   CUST0011|     TXN207175|      2023-05-23| 785.84|          credit|       2021-11-23|        high|
|   CUST0003|     TXN733052|      2023-07-14| 492.61|          credit|       2020-01-26|        high|
|   CUST0009|     TXN496922|      2023-01-23|3651.36|           debit|       2022-01-24|        high|
|   CUST0003|     TXN705397|      2023-10-10|1472.96|          credit|       2020-01-26|        high|
|   CUST0007|     TXN910620|      2023-12-27| 357.08|           debit|       2020-08-16|         low|
|   CUST0010|     TXN498591|      2023-02-10|4278.04|           debit|       2020-04-14|        high|
|   CUST0009|     TXN270555|      2023-08-21|3182.07|          credit|       2022-

Summary statistics

In [7]:
# Summary statistics
print("\nSummary Statistics:")
custrans.describe().show()


Summary Statistics:
+-------+-----------+--------------+------------------+----------------+------------+
|summary|customer_id|transaction_id|            amount|transaction_type|risk_segment|
+-------+-----------+--------------+------------------+----------------+------------+
|  count|        200|           200|               200|             200|         200|
|   mean|       NULL|          NULL|2392.4248999999995|            NULL|        NULL|
| stddev|       NULL|          NULL|1417.5792633356295|            NULL|        NULL|
|    min|   CUST0001|     TXN113266|             56.48|          credit|        high|
|    max|   CUST0020|     TXN994905|           4980.65|           debit|      medium|
+-------+-----------+--------------+------------------+----------------+------------+



Check missing values

In [8]:
# count NULL values in each column
custrans.select([(count(col(c)) - count(col(c))).alias(f"missing_{c}") for c in custrans.columns]).show()

+-------------------+----------------------+------------------------+--------------+------------------------+-------------------------+--------------------+
|missing_customer_id|missing_transaction_id|missing_transaction_date|missing_amount|missing_transaction_type|missing_account_open_date|missing_risk_segment|
+-------------------+----------------------+------------------------+--------------+------------------------+-------------------------+--------------------+
|                  0|                     0|                       0|             0|                       0|                        0|                   0|
+-------------------+----------------------+------------------------+--------------+------------------------+-------------------------+--------------------+



## Calculate the Average Transaction Amount
This block groups the joined DataFrame by customer_id and risk_segment and calculates the average amount from all transactions.

It ensures that for each customer we have computed the average of both debit and credit transactions.

In [8]:
# Calculate Average Transaction Amount per Customer
avg_df = custrans.groupBy("customer_id", "risk_segment") \
                  .agg(avg("amount").alias("avg_transaction_amount"))

# Show a sample of the average calculation
avg_df.show(10)


+-----------+------------+----------------------+
|customer_id|risk_segment|avg_transaction_amount|
+-----------+------------+----------------------+
|   CUST0010|        high|    2172.6945454545457|
|   CUST0001|         low|    2830.1476923076925|
|   CUST0019|        high|    2835.3933333333334|
|   CUST0003|        high|    2405.3107692307694|
|   CUST0004|        high|              2057.352|
|   CUST0013|         low|            2527.98375|
|   CUST0015|        high|    2167.9361538461535|
|   CUST0011|        high|    2445.7337500000003|
|   CUST0006|        high|               2144.86|
|   CUST0017|         low|              2344.725|
+-----------+------------+----------------------+
only showing top 10 rows



 All customers here are high spenders, with average amounts over 1000.

## Classify Spend Category
Based on the average transaction amount, classify each customer as follows:

low spender: if the average is less than 100

moderate spender: if the average is between 100 and 1000 (inclusive)

high spender: if the average is greater than 1000

In [9]:
# Classify Spend Category Based on Average Transaction Amount
classified_df = avg_df.withColumn("spend_category",
    when(col("avg_transaction_amount") < 100, "low spender")
    .when((col("avg_transaction_amount") >= 100) & (col("avg_transaction_amount") <= 1000), "moderate spender")
    .otherwise("high spender")
)

# Verify the new column with classification
classified_df.show(10)


+-----------+------------+----------------------+--------------+
|customer_id|risk_segment|avg_transaction_amount|spend_category|
+-----------+------------+----------------------+--------------+
|   CUST0010|        high|    2172.6945454545457|  high spender|
|   CUST0001|         low|    2830.1476923076925|  high spender|
|   CUST0019|        high|    2835.3933333333334|  high spender|
|   CUST0003|        high|    2405.3107692307694|  high spender|
|   CUST0004|        high|              2057.352|  high spender|
|   CUST0013|         low|            2527.98375|  high spender|
|   CUST0015|        high|    2167.9361538461535|  high spender|
|   CUST0011|        high|    2445.7337500000003|  high spender|
|   CUST0006|        high|               2144.86|  high spender|
|   CUST0017|         low|              2344.725|  high spender|
+-----------+------------+----------------------+--------------+
only showing top 10 rows



This table shows each customer's average transaction amount along with their risk segment.
Most customers are classified as high spenders because their average transaction amounts exceed 1000.

## Prepare the final DataFrame
Select the columns: customer_id, risk_segment, avg_transaction_amount, and spend_category.

In [10]:
# Prepare Final DataFrame with the specified columns for each customer
custrans_df = classified_df.select("customer_id", "risk_segment", "avg_transaction_amount", "spend_category")

# Show the final result
custrans_df.show(10)


+-----------+------------+----------------------+--------------+
|customer_id|risk_segment|avg_transaction_amount|spend_category|
+-----------+------------+----------------------+--------------+
|   CUST0010|        high|    2172.6945454545457|  high spender|
|   CUST0001|         low|    2830.1476923076925|  high spender|
|   CUST0019|        high|    2835.3933333333334|  high spender|
|   CUST0003|        high|    2405.3107692307694|  high spender|
|   CUST0004|        high|              2057.352|  high spender|
|   CUST0013|         low|            2527.98375|  high spender|
|   CUST0015|        high|    2167.9361538461535|  high spender|
|   CUST0011|        high|    2445.7337500000003|  high spender|
|   CUST0006|        high|               2144.86|  high spender|
|   CUST0017|         low|              2344.725|  high spender|
+-----------+------------+----------------------+--------------+
only showing top 10 rows



##  Save as Parquet

This block saves the final customer summary DataFrame in Parquet format, which is a columnar storage format optimized for performance and used in big data processing.

In [11]:
# Save the final DataFrame in Parquet format
custrans_df.write.mode("overwrite").parquet("output/final_transactions.parquet")

## Conclusion
In this analysis, we successfully processed the transaction and customer data using PySpark to gain insights into customer spending behavior. By calculating the average transaction amount for each customer, we classified them into different spend categories: low, moderate, and high spenders. The results revealed that most customers fall into the high spender category, as their average transaction amounts exceeded 1000. This classification helps to better understand customer behavior, enabling businesses to tailor their offerings or strategies accordingly. The final dataset, containing customer IDs, risk segments, average transaction amounts, and spend categories, was saved in Parquet format for efficient storage and further analysis.